# <p style="background-color:#3498DB; font-family: 'Segoe UI', Tahoma, Geneva, Verdana, sans-serif; color:#FFF; font-size:160%; text-align:center; border-radius:10px; padding:12px;">Previsões Acionárias Baseadas em Indicadores</p>

In the digital age, password security is paramount. As cyber threats evolve, understanding and reinforcing the strength of passwords becomes essential. This project aims to tackle this challenge using a data-driven approach. We leverage the "Password Strength Classifier" dataset from Kaggle, which offers a diverse set of passwords labeled by their strength. Our objective is to develop a machine learning model that accurately classifies passwords into categories of strength - weak, medium, or strong. This classification not only enhances our understanding of what constitutes a secure password but also serves as a tool for encouraging stronger cybersecurity practices. Post model development, we plan to deploy our solution using Streamlit, making it accessible and user-friendly. This project is not just about building a model; it's about contributing to safer digital environments.

**Project Structure:**

[1. **Load the Dataset:**](#1)
   
[2. **Exploratory Analysis:**](#2)

[3. **Dataset Splitting:**](#3)

[4. **Model Selection/Training:**](#4)

[5. **Model Evaluation:**](#5)
   
[6. **Model Export:**](#6)